In [1]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *

Tensorflow is not available


c:\Users\francesco\miniconda3\envs\RecSysFramework\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)
data_train_val = data_train + data_val

In [29]:
candidate_recommender_study = optuna.create_study(
    study_name="P3Beta",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
candidate_recommender = RP3betaRecommender(data_train_val, verbose=False)
candidate_recommender.fit()

[I 2023-12-31 18:17:13,106] A new study created in memory with name: P3Beta


In [30]:
n_users, n_items = data_train.shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'
training_dataframe

,ItemID
UserID,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
12633,NaN
12634,NaN
12635,NaN


In [31]:
cutoff = 50

for user_id in range(n_users):    
    recommendations = candidate_recommender.recommend(user_id, cutoff = cutoff, remove_seen_flag=True)
    training_dataframe.loc[user_id, "ItemID"] = recommendations
training_dataframe = training_dataframe.explode("ItemID")
training_dataframe

,ItemID
UserID,
0,22146
0,674
0,15344
0,587
0,2843
...,...
12637,19760
12637,16596
12637,19019


In [36]:
import scipy.sparse as sps

data_val_coo = sps.coo_matrix(data_val)
correct_recommendations = pd.DataFrame({"UserID": data_val_coo.row,
                                        "ItemID": data_val_coo.col})
training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID','ItemID'], how='left', indicator='Exist')
training_dataframe["Label"] = training_dataframe["Exist"] == "both"
training_dataframe.drop(columns = ['Exist'], inplace=True)
#training_dataframe

    UserID  ItemID
0        0      18
1        0      20
2        0      30
3        0      34
4        0      36
5        0      39
6        0      43
7        1       6
8        1      51
9        1      55
10       1      62
11       1      68
12       1      69
13       1      76
14       1      77
15       1      78
16       2      80
17       2      81
18       3      10
19       3      87
20       3      89
21       3      91
22       3      93
23       3      95
24       3      96
25       3      97
26       3     106
27       3     111
28       3     113
29       4      44
30       4     118
31       4     122
32       4     128
33       4     147
34       4     156
35       4     161
36       4     165
37       4     170
38       4     174
39       4     175
40       4     177
41       4     185
42       5      51
43       5     193
44       5     196
45       5     197
46       6     204
47       7      44
48       7     117
49       7     206
    UserID ItemID  Label      E

In [21]:
topPop = TopPop(data_train_val)
topPop.fit()

p3alpha_study = optuna.create_study(
    study_name="P3Alpha",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
p3alpha = P3alphaRecommender(data_train_val)
p3alpha.fit(**p3alpha_study.best_params)

easer_study = optuna.create_study(
    study_name="Easer",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
easer = EASE_R_Recommender(data_train_val)
easer.fit(**easer_study.best_params)

itemknncf_study = optuna.create_study(
    study_name="ItemKNNCF",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
itemknncf = ItemKNNCFRecommender(data_train_val)
itemknncf.fit(**itemknncf_study.best_params)


other_algorithms = {
    "TopPop": topPop,
    "P3alpha": p3alpha,
    "Easer": easer,
    "ItemKNNCF": itemknncf
}

[I 2023-12-31 16:33:48,133] Using an existing study with name 'P3Alpha' instead of creating a new one.


P3alphaRecommender: Similarity column 22222 (100.0%), 3058.59 column/sec. Elapsed time 7.27 sec


[I 2023-12-31 16:33:57,152] Using an existing study with name 'Easer' instead of creating a new one.


EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 3.29 min


[I 2023-12-31 16:37:32,112] Using an existing study with name 'ItemKNNCF' instead of creating a new one.


Similarity column 22222 (100.0%), 8859.04 column/sec. Elapsed time 2.51 sec


In [22]:
training_dataframe = training_dataframe.set_index('UserID')

for user_id in range(n_users):  
    for rec_label, rec_instance in other_algorithms.items():
        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()
        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)
        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list] 

training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})

item_popularity = np.ediff1d(sps.csc_matrix(data_train).indptr)
training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]
user_popularity = np.ediff1d(sps.csr_matrix(data_train).indptr)
training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]

In [23]:
training_dataframe = training_dataframe.set_index('ItemID')
training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "ItemID"})

In [24]:
groups = training_dataframe.groupby("UserID").size().values
train_recs = training_dataframe.drop(columns=["Label"])
train_labs = training_dataframe["Label"]

In [26]:
study = optuna.create_study(
    study_name="XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2023-12-31 16:38:53,559] Using an existing study with name 'XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)' instead of creating a new one.


In [27]:
def evaluator(data_train, train_recs, XGB_model, training_dataframe, data_test: sp.csr_matrix, recommendation_length: int = 10):
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = data_train.shape[0]

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = data_test.indptr[user_id]
        user_profile_end = data_test.indptr[user_id+1]

        relevant_items = data_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
        
        predict = train_recs[train_recs["UserID"] == user_id]
        predictions = XGB_model.predict(predict)
        user_ids = predict["UserID"].values
        item_ids = predict["ItemID"].values
        predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions})
        predictions = predictions.sort_values(by="Predictions", ascending=False)
        recommendations = np.array(predictions["ItemID"].values[:recommendation_length])

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped


In [ ]:
from xgboost import XGBRanker

def objective(trial):
    objective = trial.suggest_categorical("objective", ["rank:ndcg", "rank:map", "rank:pairwise"])
    n_estimators = trial.suggest_int("n_estimators", 5, 500)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    max_leaves = trial.suggest_int("max_leaves", 1, 10)
    grow_policy = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    booster = trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"])

    XGB_model = XGBRanker(
        tree_method="hist",
        objective=objective,
        n_estimators=int(n_estimators),
        random_state=None,
        learning_rate=learning_rate,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        max_depth=int(max_depth),
        max_leaves=int(max_leaves),
        grow_policy=grow_policy,
        booster=booster,
        verbosity=0, # 2 if self.verbose else 0,
    )
    XGB_model.fit(
        train_recs,
        train_labs,
        group=groups,
        verbose=False
    )
    _, _, ev_map, _, _ = evaluator(data_train, train_recs, XGB_model, training_dataframe, data_val)
    
    return ev_map

study.optimize(objective, n_trials=1000)

In [29]:
from xgboost import XGBRanker

XGB_model = XGBRanker(
    **study.best_params,
    verbosity=0, # 2 if self.verbose else 0,
    enable_categorical=True,
)
XGB_model.fit(
    train_recs,
    train_labs,
    group=groups,
    verbose=False
)

XGBRanker(base_score=None, booster='gbtree', callbacks=None,
          colsample_bylevel=None, colsample_bynode=None, colsample_bytree=None,
          device=None, early_stopping_rounds=None, enable_categorical=True,
          eval_metric=None, feature_types=None, gamma=None,
          grow_policy='lossguide', importance_type=None,
          interaction_constraints=None, learning_rate=0.09958690670411639,
          max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
          max_delta_step=None, max_depth=7, max_leaves=10,
          min_child_weight=None, missing=nan, monotone_constraints=None,
          multi_strategy=None, n_estimators=500, n_jobs=None,
          num_parallel_tree=None, random_state=None, ...)

In [30]:
def submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val):
    
    toppoprecommender = TopPop(data_train_val)
    toppoprecommender.fit()
    
    with open('data/results.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['user_id', 'item_list'])
        for user_id in users:
            predict = train_recs[train_recs["UserID"] == user_id]
            user_ids = predict["UserID"].values
            item_ids = predict["ItemID"].values
            predictions = XGB_model.predict(predict)
            if not list(predictions):
                predictions = toppoprecommender.recommend(0, cutoff = 10)
                item_list = [itemmap[i] for i in predictions]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                continue
            predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions.ravel()})
            predictions = predictions.sort_values(by="Predictions", ascending=False)
            recommendations = np.array(predictions["ItemID"].values[:10])
            item_list = [itemmap[i] for i in recommendations]
            writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                

In [31]:
submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val)